<a href="https://colab.research.google.com/github/Awnder/rag-workshop/blob/main/rag-workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries

In [ ]:
#https://streamlit.io/
# used andrew's pinecone and groq api keys, with github access key from sbc

In [3]:
%pip install langchain langchain-community langchain-pinecone
%pip install openai groq pinecone-client unstructured[all-docs] unstructured_inference sentence-transformers docx2txt openpyxl onnx==1.16.1
%pip install pdfminer==20191125 pdfminer.six==20221105 pillow_heif pi-heif

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.document_loaders import PyPDFLoader, UnstructuredExcelLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredImageLoader, GithubFileLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import openai
import os
from dotenv import load_dotenv
from groq import Groq

c:\Users\andrew.shiroma\My Drive\R&I AI Chatbot\venv_r&i_assistant\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Initialize the HuggingFace Embeddings client

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\andrew.shiroma\AppData\Local\Temp\ipykernel_22776\619240441.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


# Load in the Data

In [15]:
load_dotenv()

# need to install tesseract in order to use the OCR (optical character recognition) image features
# https://github.com/langchain-ai/langchain/issues/9644

def process_directory(directory_path):
    ''' resursively processes all files in a directory '''
    data = []
    # other docs
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
                data.append(build_directory_document(file_path, loader))
            # elif file[-3:] == 'png':
            #     loader = UnstructuredImageLoader(file_path)
            #     data.append({"File": file_path, "Data": loader.load()})
            elif file.endswith(".docx"):
                loader = Docx2txtLoader(file_path)
                data.append(build_directory_document(file_path, loader))
            elif file.endswith(".xlsx"):
                loader = UnstructuredExcelLoader(file_path)
                data.append(build_directory_document(file_path, loader))

    return data

def build_directory_document(file_path: str, loader: object):
    ''' create a Document object with metadata and page content '''
    return Document(
        metadata={
            "source": file_path
        },
        page_content=f"Source: {file_path}\n{loader.load()[0].page_content}"
    )

def process_github():
    ''' recursively processes all files in a github repo '''
    data = []
    loader = GithubFileLoader(
        repo="reportingandinsights/common-code",
        branch="main",
        access_token=os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
        file_filter=lambda file_path: file_path.endswith(".md") or file_path.endswith(".txt") or file_path.endswith(".xml"),
    )
    
    repo_name = "reportingandinsights/common-code/"
    for gitdoc in loader.load():
        data.append(build_github_doc(repo_name, gitdoc))

    return data

def build_github_doc(repo_name: str, gitdoc: list):
    ''' create a Document object with metadata and page content. github loader loads all documents so requires a different function'''
    return Document(
        metadata={
            "source": f"{repo_name}{gitdoc.metadata['path']}"
        },
        page_content=f"Source: {repo_name}{gitdoc.metadata['path']}\n{[gitdoc][0].page_content}" # convert the doc to a 1-element list to access the page_content
    )

directory_path = "./r&i_assistant_docs"
documents = []
documents += process_directory(directory_path)
documents += process_github()


In [16]:
documents

[Document(metadata={'source': './r&i_assistant_docs\\Data Dictionaries Guide.pdf'}, page_content='Source: ./r&i_assistant_docs\\Data Dictionaries Guide.pdf\nV i e w D a t a D i c t i o n a r i e s \n •Whatlevelofdetaildoyouneed?1 . B u s i n e s s l o g i c f o r a m e t r i c \n ● Sheet:KPIDictionary○ metric-levelinfoforreportingneedforHCMinistryApp&reportingneeds;lackscolumninfo.○ R&IKPIDictionary\n2 . D a t a c o l l e c t i o n / d a t a e n t r y \n ● Sheet:R&IKPIDictionary>>column"DataEntryProcess"○ metric-levelinfoforreportingneedforHCMinistryApp&reportingneeds;lackscolumninfo.○ R&IKPIDictionary\n3 . U s e r - f a c i n g r e p o r t n o t e s \n ● File:R&IKPIDictionary.xlsx>>Sheet:ExplanatoryNotesforReports○ ExplanatoryNotesforReports○ Whenthereispotentiallyambiguouschurch-centriclanguage,thetechnicaltermsareprovided\n4 . T a b l e s o n l y ( a n d P o w e r B I D a t a f l o w l o c a t i o n ) ● DataflowListing○ Table-levelinfoforcustomandDWHandHCtables;○ DataflowListing\n5 

# Inserting Data Into Pinecone

In [17]:
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "ri-assist"

namespace = "documentation"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [20]:
# for idx, document in enumerate(documents):
#     print("Processing document: ", idx)
#     vectorstore_from_documents = PineconeVectorStore.from_documents(
#         [document],
#         embeddings,
#         index_name=index_name,
#         namespace=namespace
#     )

# add all documents at once. this seems to be faster than the above
vectorstore.add_documents(documents=documents) 

['48d65a82-e61c-41b9-965b-7c0cef923400',
 '7dc58f2e-b5dd-45b0-8e3d-25a3bde5de38',
 '5410d4df-8151-44de-a2c6-017528b0d3b7',
 '2fcfe5f0-e78c-4517-a459-68dec2c2ee2f',
 '4ed4cd97-2cda-480e-ad88-b7aaf21cd32e',
 '5b931356-12f3-4979-8b5a-418b51969360',
 'c1fbb0b6-bc4a-4cd2-aac4-c001f7fe1b67',
 '5cdd9349-f318-4b45-9ecb-988754bf548b',
 'ca93da95-7a07-4d5a-9e54-cf0ee919d003',
 'd7f96e65-b33d-438b-bdea-2827051ac123',
 'dcc4b01e-f53e-44c9-80cd-5de22196bcca',
 '5cd2558b-02bb-43d2-a533-fec0671c3f40',
 'e68a14de-e80d-4d1b-96d1-fa35a367c249',
 'e81f2a94-86e5-46d3-af1d-35878344b8c2',
 '43da74db-4538-4fed-8fd3-937c43ec02d1',
 '7bcc8b50-aaaa-4b07-9c84-6b238bf45c25',
 '58398934-0e18-42b4-8150-b53cc7c0b240',
 '8a78cd6a-04cf-4651-b6f8-57b5ed4e8b34',
 'b2689aba-1f07-4d08-803b-cd33d4c3716d',
 '75106ccf-af54-4f25-a5ff-b568a87518f4',
 '7f7500f3-a427-496b-b825-e89360dc480d',
 '191c1c8e-1333-4a42-ac4b-9a563b8eb653',
 'a90fff99-57bd-48ae-a6d2-ebc468bd05c8',
 '425ecdbd-41f3-4d91-bf97-c43eabafb298',
 '5faf9f24-d817-

In [ ]:
# doesnt seem to work as intended. I think dramatically writes to pinecone way more than necessary
# import multiprocessing

# def add_documents_batch(batch):
#     """Adds a batch of documents to the Pinecone index."""
#     vectorstore.add_documents(batch)

# def create_multiprocess_pool(documents, num_processes=None):
#     if num_processes is None:
#         num_processes = multiprocessing.cpu_count() # use all available cores
    
#     batch_size = len(documents) // num_processes
#     batches = [documents[i:i+batch_size] for i in range(0, len(documents), batch_size)]

#     with multiprocessing.Pool(processes=num_processes) as pool:
#         pool.map(add_documents_batch, batches) # requires .map(function, iterable)
#         print('pooling: ' + batches)

# create_multiprocess_pool(documents)

# BATCH_SIZE = 1
# for i in range(0, len(documents), BATCH_SIZE):
#     batch = documents[i:i+BATCH_SIZE]
#     vectorstore.add_documents(batch)
#     print(f"Inserting batch {i // BATCH_SIZE + 1} of {len(documents) // BATCH_SIZE + 1}")

# Initialize the Groq client

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)

In [22]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

groq_client = Groq(api_key=groq_api_key)

# Perform RAG

In [23]:
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [32]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [37]:
query = "how to use calculate in power bi?"

In [ ]:
# construct query, embed it, get pinecone vector match, and grab contexts

# running into some issues with no matching with the new vector store method. its bc added documents to ( Default ) namespace (see pinecone)
query_embed = get_huggingface_embeddings(query)
top_matches = pinecone_index.query(vector=query_embed.tolist(), top_k=5, include_metadata=True, namespace=namespace)
contexts = [item['metadata']['text'] for item in top_matches['matches']]
print(contexts)

documentation
[]


In [29]:
# creating a formatted query for the AI
augmented_query = "<CONTEXT>\n" + "\n-------\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\nMY QUESTION:\n" + query

In [30]:
print(augmented_query)

<CONTEXT>

-------
</CONTEXT>

MY QUESTION:
how to conditionally hide unwanted visuals?


In [132]:
system_prompt = f'''
You are an expert at understanding and explaining business technology and development, including Scrum methodology, Power BI development, and other documentation.
Answer any questions based on the data provided. Always consider the context of the question and provide the most accurate and relevant information when forming a response.
If unsure of anything, mention it in the response and provide a suggestion for further research. Cite your sources used below your response.'''

llm_response = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",  "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [133]:
print(response)

Based on the context provided, `CALCULATE` in Power BI is used to calculate an aggregate value using the specified filter, grouping, and columns.

The general syntax of `CALCULATE` is as follows:

```dax
CALCULATE (
    <expression>,
    <filter1>,
    <filter2>,
    ...
)
```

Where `<expression>` is the calculation you want to perform, and `<filter1>`, `<filter2>`, etc. are the filters you want to apply to the data.

In the context provided, several examples of using `CALCULATE` are shown:

1. **Using `CALCULATE` with `MAX`**

```dax
C_Phone_RowHeader = CALCULATE(
  MAX('FormCard_Response_Summary'[Answer_Value]),
  OR('FormCard_Response_Summary'[Custom_Question_Name] = "Cell Phone",
    'FormCard_Response_Summary'[Custom_Question_Name] = "cell_phone")
)
```

In this example, `CALCULATE` is used with the `MAX` function to get the maximum value of `Answer_Value` from the `FormCard_Response_Summary` table, filtered by rows where `Custom_Question_Name` is either "Cell Phone" or "cell_pho

# Putting it all together